In [1]:
import numpy as np
from scipy.signal import butter
from scipy.signal import lfilter
import pandas as pd
import datetime as dt
import csv
from numpy import dot
import threading
import time
import os
import json
from dateutil.parser import parse

In [2]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band', analog=False)
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def normalization(lpf):
    df = pd.DataFrame(lpf)
    norm_df = (df - df.mean()) / df.std()
    return norm_df

def time_transe(time):
    if time[0:2] == '오전':
        time = time[3:]
        t_date = parse(time)
    else:
        h = int(time[3:4])+12
        time = time[4:]
        t_date = parse(str(h)+time)
    return t_date

def loadRawdata(path):
#     path = 'C:/MAVE_RawData/2021-06-05_오후 5_18/Rawdata.txt'
    # path2 = 'C:/MAVE_RawData/오재무/2021-06-04_오후 5_52/Rawdata.txt'
#     print(path)
    temp = pd.read_csv(path,delimiter='\t',encoding='cp949')
    # temp2 = pd.read_csv(path2,delimiter='\t',encoding='cp949')
    t_fp1 = temp['Time']
    e_fp1 = temp['EEG_Fp1']
    # t_fp2 = temp2['Time']
    # e_fp2 = temp2['EEG_Fp1']
    return t_fp1, e_fp1, temp

In [ ]:
with open('C:\\Users\\toddl\\OneDrive\\바탕 화면\\ERP_TEST_PANNEL_SET\\ERP_num1\\ERP_Num1_Data\\timeData.json', 'r') as f:
    json_data = json.load(f)

path = '0606_정수_rd/Rawdata.txt'
t_fp1, e_fp1, fp1 = loadRawdata(path)

In [ ]:
x = range(0,len(e_fp1))
Fs = 512
lowcut = 0.5 #0.5
highcut = 30 #30
m_lpf = butter_bandpass_filter(e_fp1,lowcut, highcut, Fs, order=5)


# 그래프 그리는 부분
# N = len(e_fp1)
# k = np.arange(N)

# T = N/Fs
# freq = k / T
# freq = freq[range(int(N/2))]

# yfft = np.fft.fft(e_fp1)
# yf = yfft / N
# yf = yf[range(int(N/2))]

# # plt.axhline(color='black')
# plt.plot(x, e_fp1, 'b', label='origin')
# plt.show()

# plt.plot(freq,abs(yf),'gray')
# plt.axvline(x = 30, color = 'red', alpha = 0.2)
# plt.show()

In [ ]:
#파일 생성
df = pd.DataFrame(m_lpf)
s_df=pd.concat([t_fp1, df], axis=1)
print(s_df)
# .to_csv 
# 최초 생성 이후 mode는 append
# 'C:/MAVE_RawData/2021-06-02_오후 2_53/Rawdata.txt'
w_path = 'C:/MAVE_RawData/2021-06-04_오후 6_16/filtered.csv'
# w_path = 'C:/MAVE_RawData/'+t_date+'/filtered.csv'

# if not os.path.exists(w_path):
#     s_df.to_csv(w_path, index=False, mode='w', encoding='utf-8-sig')
# else:
#     s_df.to_csv(w_path, index=False, mode='a', encoding='utf-8-sig', header=False)

In [ ]:
time_str = ''
peek_time = ''
start_index = ''
def checkPeekTime(json_data):
    peek = ''
    while(True):
        if len(json_data['peekTime'])!=0:
            peek = json_data['peekTime'][-1]
            break
        else:
            continue
    return peek

peek_time = checkPeekTime(json_data)
# for i in json_data['peekTime']:
#     if i != '':
#         peek_time = i
#         continue
#     else:
#         break

peek_time = parse(peek_time[11:-3])
print('peektime > {}'.format(peek_time))
# m_lpf
for k,v in t_fp1.items():
#     if peek_time >
    time_str = time_transe(v)
    if peek_time >= time_str:
        if peek_time - time_str < dt.timedelta(milliseconds = 200):
            print("123"+k)
            start_index = k
            break
        else:
            print("123-else")
            continue
    else:
        if time_str - peek_time < dt.timedelta(milliseconds = 200):
            print("321"+k)
            start_index = k
            break
        else:
            print("321-else")
            continue

In [ ]:
#더해주는 값 테스트마다 바꿔주기
end_index = start_index+320

extraction_eeg = m_lpf[start_index:end_index]
#정규화
norm_e = (extraction_eeg - extraction_eeg.min()) / (extraction_eeg.max() - extraction_eeg.min())
#밑에는 N200과 P300을 어떻게 해서 반응을 할 지 정해야함.

if norm_e.max()>=0.65:
    if norm_e.min()<=0.3:
        print('You think 4 !')
    else:
        print('fail condition 2')
else:
    print('fail condition 1')
